<H2>Create a single span bridge deck - v21.1 and later</H2>

In [413]:
import numpy as np
import math
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.22.0") # Connect to LUSAS Modeller

# If there is not currently a model open, create one
if not lusas.existsDatabase():
    lusas.newProject("Structural", "Bridge Deck")

db = lusas.database() # Reference to the model database for convenience

# Close any previous results
db.closeAllResults()

# Delete all previous model data
db.deleteAll()
db.deleteAllAttributes()
db.deleteAllAnalyses()
db.deleteAllNoGroups()

# 3D model with Z vertical
db.setAnalysisCategory("3D")

# Set the unit system
db.setModelUnits("kN,m,t,s,C")

Define model parameters

In [414]:
deck_width = 18
deck_length = 25
no_beams = 4
deck_overhang = 1.5
skew_angle = 35
deck_thk = 0.2

mesh_size = 1


Create the attributes to be assigned to the geometric features. Mesh, Geometric and Material.

In [415]:
# Deck surface attributes
deck_mesh_attr    = db.createMeshSurface("SMsh1").setRegularSize("QTS4", mesh_size, True)
deck_surface_attr = db.createGeometricSurface("Deck Slab").setSurface(deck_thk, 0.0)
deck_mat_attr     = db.createIsotropicMaterial("Concrete deck", 34.8E6, 0.2, 2.4)

# Beam mesh attributes
beam_mesh_attr = db.createMeshLine("LMsh2").setSize("BMI21", mesh_size)
beam_geom_attr = db.createGeometricLine("LGeo2").setFromLibrary("Australian Sections", "Precast Super T (Open)", "T3 (100)", 0, 0, 0)
beam_geom_attr.setValue("elementType", "3D Thick Beam")
beam_geom_attr.setEccentricityOrigin("Centroid", "Fibre", "", "A1")
beam_mat_attr = db.createIsotropicMaterial("Concrete beam", 34.8E6, 0.2, 2.4)

# Support attributes
support_pinned_attr = db.createSupportStructural("Pinned").setStructural("R", "R", "R", "F", "F", "F", "F", "F", "C", "F") # F=Free, R=Restrained
support_slide_attr  = db.createSupportStructural("Slide").setStructural("F", "R", "R", "F", "F", "F", "F", "F", "C", "F")


In order to assign any attributes we need a loadcase to assign them to. <br>We'll create a base analysis with a single loadcase

In [416]:
base_analysis = db.createAnalysisStructural("Base")
# We'll get the loadcase and add automatic gravity to it
# NOTE: getLoadset returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
win32.CastTo(db.getLoadset("Loadcase 1", 0), "IFLoadcase").addGravity(True) 

Define some useful helper functions for creating lines. This keeps the code clean and avoid duplication

In [417]:
# Define a useful helper function to create a line from two point coordinates
def create_line(p1:list, p2:list) -> 'IFLine':
        # geometryData object contains all the settings to perform a geometry creation
        geomData = lusas.geometryData().setAllDefaults()  
        # set the options for creating straigt lines from coordinates
        geomData.setCreateMethod("straight")        
        geomData.setLowerOrderGeometryType("coordinates")        
        
        # Add the cordinates, lines directions will follow the order of the coordinates
        geomData.addCoords(p1[0], p1[1], p1[2])    # Set the coordinates of the first point X,Y,Z
        geomData.addCoords(p2[0], p2[1], p2[2])    # Set the coordinates of the second point X,Y,Z

        # Create the line, get the line objects array from the returned object set and return the 1 and only line
        return db.createLine(geomData).getObjects("Line")[0]


# Define a useful helper function to create a line from two point objects
# Note that we expect two IFPoint objects, these are references to points already created in the model
def create_line_from_points(p1:'IFPoint', p2:'IFPoint') -> 'IFLine':
        # geometryData object contains all the settings to perform a geometry creation
        geomData = lusas.geometryData().setAllDefaults()         
        # set the options for creating straigt lines from points
        geomData.setCreateMethod("straight")        
        geomData.setLowerOrderGeometryType("points")

        # Create an object set to contain the points and use this set to create the line
        obs = lusas.newObjectSet()                 
        obs.add(p1)
        obs.add(p2)

        # Create the line, get the line objects array from the returned object set and return the 1 and only line
        return obs.createLine(geomData).getObjects("Line")[0]

Helper function to calculate beam coordinates accounting for the skew angle

In [418]:
def get_x(y:float)->float:
    return y * math.tan(math.radians(skew_angle))

Calculate the x and y coordinates of the frame grid

In [419]:
# Calculate the beam spacing 
beam_spacing = (deck_width - 2*deck_overhang) / (no_beams-1)
print(f"Beam Spacing = {beam_spacing}")
assert beam_spacing > 0, "Cant have a negative beam spacing"

# Y coordinates of lomngitudinal lines. 
# Note the surfaces between the beams are split evenly between the beams
long_ys = [0.0, deck_overhang]
for i in range(0, no_beams-1):
    long_ys.append(long_ys[-1] + beam_spacing/2)
    long_ys.append(long_ys[-2] + beam_spacing)
long_ys.append(deck_width)
print(f"long line y coordinates = {long_ys}")

# Create the longitudinal lines storing them in two lists for reference
# One for all lines and one for just the beam lines
longitudinal_lines = []
beam_lines = []

# Create the longitudinal lines. ALtenate lines are beam lines so we'll assign the beam attributes only to those
for i, y in enumerate(long_ys):
    line = create_line([get_x(y), y, 0], [get_x(y) + deck_length, y, 0])
    if i%2 != 0:
        beam_mesh_attr.assignTo(line)
        beam_geom_attr.assignTo(line)
        beam_mat_attr.assignTo(line)
        beam_lines.append(line)
    longitudinal_lines.append(line)

# Now the start end support lines
end1_lines = []
for i in range(0, len(longitudinal_lines)-1):
    end1_lines.append(create_line_from_points(longitudinal_lines[i].getStartPoint(), longitudinal_lines[i+1].getStartPoint()))

# And opposite end support lines
end2_lines = []
for i in range(0, len(longitudinal_lines)-1):
    end2_lines.append(create_line_from_points(longitudinal_lines[i].getEndPoint(), longitudinal_lines[i+1].getEndPoint()))


Beam Spacing = 5.0
long line y coordinates = [0.0, 1.5, 4.0, 6.5, 9.0, 11.5, 14.0, 16.5, 18]


Define helper function for creating surfaces.

In [420]:
def create_surface_from_lines(lines: list):
    # Set the options for creating surfaces from lines
    geometryData = lusas.geometryData().setAllDefaults()
    geometryData.setLowerOrderGeometryType("lines")

    # Add the lines being used to define the surface to an object set
    obs = lusas.newObjectSet()
    for p in lines:
        obs.add(p)

    # Use the object set to create the surface and then get the created features as an array. (in this case an array of 1) 
    objs = obs.createSurface(geometryData).getObjects("Surface")

    # Assign the mesh attribute to the created surfaces
    deck_mesh_attr.assignTo(objs)
    # Assign the geometric thickness attribute to the created surfaces
    deck_surface_attr.assignTo(objs)
    # Assign the material attribute to the created surfaces
    deck_mat_attr.assignTo(objs)

Create surfaces between the deck lines and assign the mesh and thickness attributes

In [421]:
for i in range(0, len(longitudinal_lines)-1):
    # Create a list of lines in clock-wise order from which to create the surface
    lines=[]
    lines.append(longitudinal_lines[i])
    lines.append(end1_lines[i])
    lines.append(end2_lines[i])
    lines.append(longitudinal_lines[i+1])
    create_surface_from_lines(lines)

Generate the mesh from the assignments

In [422]:
db.resetMesh()
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x2031457759072>

Assign the support attributes to the points at the beam ends

In [423]:
for i in range(1, len(beam_lines)-1):
    support_pinned_attr.assignTo(beam_lines[i].getStartPoint())
    support_slide_attr.assignTo(beam_lines[i].getEndPoint())

Create a group for each of the beams with the associated deck

In [424]:
for i, line in enumerate(beam_lines):
    group = db.createGroup(f"Beam {i+1}")
    group.add(line)
    group.add(line.getHOFs())

Create staged analyses with deactivated deck

In [ ]:
# Deactivate attribute disables elments to which its assigned
deactive_attr = db.createDeactivate("Deact1").setDeactivate("activeMesh", 100.0, 1.0E-6)
# Activate attribute reactivates attributes in the loadcase to which it's assigned
activate_attr = db.createActivate("Act1")

# Create 3 separate analyses, 1st stage is for the beams, 2nd stage is long term and the 3rd stage is short term
stage1_analysis = db.createAnalysisStructural("Stage 1", False)
stage2_analysis = db.createAnalysisStructural("Stage 2", False)
stage3_analysis = db.createAnalysisStructural("Stage 3", False)

# We'll deactivate the deck in stage 1 only.
# The deactiavte attribute must be assigned to the first loadcase in the analysis, so we'll create that first
stage1_loadcase = db.createLoadcase("Beam Self Weight", "Stage 1")
# To assign the attribute to the correct loadcase we need to provide additional infomation
# We do this with the assignment object
assignment = lusas.assignment().setAllDefaults()
assignment.setLoadset(stage1_loadcase)
# Now we need to get the objects to assign to. In this case we know that all surfaces are to be deactivated
surface_set = db.getObjects("Surfaces")
# Now assign the attribute to the surfaces with the additional assignment object
deactive_attr.assignTo(surface_set, assignment)
# Set gravity on the beams only loadcase
stage1_loadcase.addGravity(True)

Calculate and assign loads for the wet concrete of th edeck in stage 1

In [426]:
# Helper function to get the area of all surfaces attached to the given line
def get_attached_surface_area(line) -> float:
    surface_area = 0
    for obj in line.getHOFs():
        if obj.getTypeCode() == 4: # Surface
                surface_area += obj.getArea()
    return surface_area

In [427]:
# Create a loadcase for the wet concrte loads to be applied to the beams in stage 1
wet_conc_loadcase = db.createLoadcase("Wet Concrete", "Stage 1")
# Set the loadcase in the assignment object
assignment = lusas.assignment().setAllDefaults()
assignment.setLoadset(wet_conc_loadcase)

# Now calculate the deck area for each beam based on the attached surfaces
for i, line in enumerate(beam_lines):
    deck_area = get_attached_surface_area(line)
    beam_load = (deck_area * deck_thk * 2.4)/line.getLineLength()
    # And create a load attribute for each beam.
    attr = db.createLoadingBeamDistributed(f"Wet Concrete - Beam {i+1}")
    attr.setBeamDistributed("Parametric", "Global", "beam")
    attr.addRow(0.0, 0.0, 0.0, -beam_load, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -beam_load, 0.0, 0.0, 0.0)
    # and assign the load
    attr.assignTo(line, assignment)

# set the wet concrete loadcase active
lusas.view().setActiveLoadset(wet_conc_loadcase)

Solve the analyses

In [428]:
# Solve each analysis and open all the available results files
db.getAnalysis("Base").solve(True)
db.getAnalysis("Stage1 1").solve(True)
db.openAllResults(False)

Create beam/shell slicing to compute equivalent beam results. Slices are positioned at 2m increments for demo

In [429]:
slice_type = 4      # Constant spacing
slice_position = "2"
#for i, line in enumerate(beam_lines):
    #slice1 = db.createBeamShellSlice("Girder 1")
    #slice1.setBeamShellSlice(slice_type, slice_position, -1, 0.0, -1, -1, beam_spacing, 0, -1, "Slice", 0, "", "2", 0.0)
slice2 = db.createBeamShellSlice("Girder 2")
slice2.setBeamShellSlice(slice_type, slice_position, -1, 0.0, -1, -1, beam_spacing, 0, -1, "Slice", 0, "", "3", 0.0)

Plot diagrams of the beam shell resultants

In [430]:
if 0:
    lusas.view().insertDiagramsLayer()
    diagrams_layer = lusas.view().diagrams()
    diagrams_layer.setResultsTransformNone()
    diagrams_layer.setResults("Beam/Shell Slice Resultants", "My")
    diagrams_layer.setLocation("BeamShellSlice")
    diagrams_layer.setAssocOption("Single")


Create results tables known as "Print Results Wizards"

In [431]:
attr = db.createPrintResultsWizard("PRW1")

#attr.setResultsType("Components")
attr.setResultsOrder("Mesh")
attr.setResultsContent("Tabular")                       # Tabular results only, summary not required
attr.setResultsEntity("Beam/Shell Slice Resultants")
attr.setExtent("Elements showing results", "")
attr.setResultsLocation("BeamShellSlice")

# Specify the loadcases required.
attr.setLoadcasesOption("Selected")
lcIDs = [1]
# We need to pass corresponding array of results ids
lcResFileIDs = [0] *len(lcIDs)
lcEigenvalueIDs = [-1] *len(lcIDs) # No eigenvalues hence -1
lcHarmonicIDs = [-1] *len(lcIDs)   # No harmonic loadcases hence -1
attr.setLoadcases(lcIDs, lcResFileIDs, lcEigenvalueIDs, lcHarmonicIDs)

# Results components to report
components = ["Fx", "Fy", "Fz", "Mx", "My", "Mz"]
attr.setComponents(components)

attr.showCoordinates(True) # Add the coordinates of each slice location for use with design checks, plots etc

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFPrintResultsWizard instance at 0x2031434455024>